In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

foldername = '/content/drive/My Drive/trans/transformer-glucose/'
assert foldername is not None, "[!] Enter the foldername."

# add path to .py code
import sys
sys.path.append(foldername)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjJRqv3q8RjU9_kYlpcKrGaiFwtKCbE_NhqgTwDh6DhPiR6GwgzD0I


In [ ]:
# load libraries
import numpy as np
import pandas as pd
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from gluformer.attention import *
from gluformer.encoder import *
from gluformer.decoder import *
from gluformer.embed import *
from gluformer.model import *

from gludata.data_loader import *
from utils.train import *
from utils.test import *

In [ ]:
# load data
PATH = foldername + '/gludata/data/'
BATCH_SIZE = 32
LEN_PRED = 12
LEN_LABEL = 60
LEN_SEQ = 180

train_data = CGMData(PATH, 'train', [LEN_SEQ, LEN_LABEL, LEN_PRED])
train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=True)

val_data = CGMData(PATH, 'val', [LEN_SEQ, LEN_LABEL, LEN_PRED])
val_data_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=True)

test_data = CGMData(PATH, 'test', [LEN_SEQ, LEN_LABEL, LEN_PRED])
test_data_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=True)

In [ ]:
LEN_PRED_MODEL = 12
LEN_LABEL = 60
LEN_SEQ = 180

# define model
D_MODEL = 512
D_FCN = 2048
N_HEADS = 8
R_DROP = 0.05
ACTIV = "relu"
NUM_ENC_LAYERS = 2
NUM_DEC_LAYERS = 1
DISTIL = True
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = Gluformer(d_model=D_MODEL, n_heads=N_HEADS, d_fcn=D_FCN, r_drop=R_DROP, activ=ACTIV, 
                 num_enc_layers=NUM_ENC_LAYERS, num_dec_layers=NUM_DEC_LAYERS,
                 distil=DISTIL, len_pred=LEN_PRED_MODEL)
model = model.to(DEVICE)

In [ ]:
# define loss and optimizer
lr = 0.0002
criterion =  nn.MSELoss()
model_optim = torch.optim.Adam(model.parameters(), lr=lr, betas=(0, 0.9))

In [ ]:
UPPER = 402
LOWER = 38
SCALE_1 = 5
SCALE_2 = 2

# define params for training
PATH_MODEL = foldername + "/model_best.pth"
EPOCHS = 100
TRAIN_STEPS = len(train_data_loader)
early_stop = EarlyStop(20, 0)

for epoch in range(EPOCHS):
  iter_count = 0
  train_loss = []
  
  model.train()
  epoch_time = time.time()
  curr_time = time.time()
  
  for i, (subj_id, batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(train_data_loader):
    iter_count += 1
    # zero-out grad
    model_optim.zero_grad()

    pred, true = process_batch(subj_id = subj_id, 
                               batch_x=batch_x, 
                               batch_y=batch_y, 
                               batch_x_mark=batch_x_mark, 
                               batch_y_mark=batch_y_mark, 
                               len_pred=LEN_PRED, 
                               len_label=LEN_LABEL, 
                               model=model, 
                               device=DEVICE)
    loss = criterion(pred, true)
    train_loss.append(loss.item())

    # print every 100
    if (i+1) % 100==0:
      print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
      speed = (time.time() - curr_time) / iter_count
      left_time = speed * ((EPOCHS - epoch) * TRAIN_STEPS - i)
      print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
      iter_count = 0
      curr_time = time.time()
      
    loss.backward()
    model_optim.step()

  # compute average train loss
  train_loss = np.average(train_loss)

  # compute validation loss
  model.eval()
  val_loss = []
  for i, (subj_id, batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(val_data_loader):
    pred, true = predict_batch(subj_id = subj_id, 
                               batch_x=batch_x, 
                               batch_y=batch_y, 
                               batch_x_mark=batch_x_mark, 
                               batch_y_mark=batch_y_mark, 
                               len_pred=LEN_PRED, 
                               len_pred_model=LEN_PRED_MODEL,
                               len_label=LEN_LABEL, 
                               model=model, 
                               device=DEVICE)
    pred = pred.detach().cpu().numpy(); true = true.detach().cpu().numpy()
    # transform back
    pred = (pred + SCALE_1) / (SCALE_1 * SCALE_2) * (UPPER - LOWER) + LOWER
    true = (true + SCALE_1) / (SCALE_1 * SCALE_2) * (UPPER - LOWER) + LOWER
    # compute APE
    ape_val = np.mean(np.abs((pred - true)) / np.abs(true))
    val_loss.append(ape_val)
  val_loss = np.median(val_loss)
  
  # compute test loss
  test_loss = []
  test_loss_3 = []
  for i, (subj_id, batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_data_loader):
    pred, true = predict_batch(subj_id = subj_id, 
                               batch_x=batch_x, 
                               batch_y=batch_y, 
                               batch_x_mark=batch_x_mark, 
                               batch_y_mark=batch_y_mark, 
                               len_pred=LEN_PRED, 
                               len_pred_model=LEN_PRED_MODEL,
                               len_label=LEN_LABEL, 
                               model=model, 
                               device=DEVICE)
    pred = pred.detach().cpu().numpy(); true = true.detach().cpu().numpy()
    # transform back
    pred = (pred + SCALE_1) / (SCALE_1 * SCALE_2) * (UPPER - LOWER) + LOWER
    true = (true + SCALE_1) / (SCALE_1 * SCALE_2) * (UPPER - LOWER) + LOWER
    # compute APE: 15 mins (3 points)
    ape_3 = np.mean(np.abs((pred[:, :3, :] - true[:, :3, :])) / np.abs(true[:, :3, :]))
    # compute APE: full
    ape_12 = np.mean(np.abs((pred - true)) / np.abs(true))
    test_loss_3.append(ape_3)
    test_loss_12.append(ape_12)
  test_loss_3 = np.median(test_loss_3)
  test_loss_12 = np.median(test_loss_12)
  model.train()

  # check early stopping
  early_stop(val_loss, model, PATH_MODEL)
  if early_stop.stop:
    print("Early stopping...")
    break

  # update lr
  # adjust_learning_rate(model_optim, epoch, lr)
  
  print("Epoch: {} cost time: {}".format(epoch+1, time.time() - epoch_time))
  print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Val Loss: {3:.7f} Test Loss (15 mins): {4:.7f} Test Loss (full): {5:.7f} ".format(
      epoch + 1, TRAIN_STEPS, train_loss, val_loss, test_loss_3, test_loss_12))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Выходные данные были обрезаны до нескольких последних строк (5000).
	speed: 0.0759s/iter; left time: 62486.6675s
	iters: 6600, epoch: 3 | loss: 0.4209405
	speed: 0.0759s/iter; left time: 62473.8149s
	iters: 6700, epoch: 3 | loss: 0.6992296
	speed: 0.0758s/iter; left time: 62433.2401s
	iters: 6800, epoch: 3 | loss: 1.3063357
	speed: 0.0759s/iter; left time: 62482.8004s
	iters: 6900, epoch: 3 | loss: 0.5526559
	speed: 0.0758s/iter; left time: 62432.1851s
	iters: 7000, epoch: 3 | loss: 0.6058303
	speed: 0.0759s/iter; left time: 62483.5275s
	iters: 7100, epoch: 3 | loss: 0.6598281
	speed: 0.0759s/iter; left time: 62460.5090s
	iters: 7200, epoch: 3 | loss: 0.5011334
	speed: 0.0758s/iter; left time: 62407.0427s
	iters: 7300, epoch: 3 | loss: 1.0131158
	speed: 0.0759s/iter; left time: 62439.0240s
	iters: 7400, epoch: 3 | loss: 0.7728500
	speed: 0.0759s/iter; left time: 62437.0521s
	iters: 7500, epoch: 3 | loss: 0.3767430
	speed: 0.0759s/iter; left time: 62429.6614s
	iters: 7600, epoch: 3 | lo

In [ ]:
torch.save(model.state_dict(), foldername + "model_last.pth")